In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import regex
import re
import seaborn as sn
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from textblob import Word
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score,roc_curve,auc
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from tqdm import tqdm
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.preprocessing import LabelEncoder,LabelBinarizer
from sklearn.metrics import classification_report
import fastText
plt.style.use('ggplot')
%matplotlib notebook

In [2]:
def fonction_nettoyage_text(df):
    # supprission des ponctuations
    rm_ponct = str.maketrans('','',string.punctuation)
    df = df.apply(lambda x:x.translate(rm_ponct))
    # suppression les unicodes
    df = df.apply(lambda x:x.encode("ascii","ignore").decode("utf-8"))
                                                
    # suppression des URLs
    df = df.apply(lambda x:re.sub(r'http\S+',"",x))
    
    # suppression des stopwords
    stop_en = stopwords.words("english")
    df = df.apply(lambda x:" ".join(x.lower() for x in np.str(x).split() if x.lower() not in stop_en))
                                                  # Lemmatisation
    df = df.apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))
                  
    return(df)

In [3]:
# import des données
dataset = pd.read_csv("data_set_version_final.csv")
dataset.resume=fonction_nettoyage_text(dataset.resume)

## Split Train Test:

In [4]:
X_train,X_test,Y_train,y_test = train_test_split(dataset.resume,dataset.Labels,test_size=0.3,shuffle=True)
x_train,x_val,y_train,y_val = train_test_split(X_train,Y_train,test_size=0.3,shuffle=True)

###  Exporter les données train /test/val:

In [5]:
x_train.to_csv("FastTestFolder/xtrain.txt",index=False)
x_val.to_csv("FastTestFolder/xval.txt",index=False)
X_test.to_csv("FastTestFolder/Xtest.txt",index=False)

/home/tantely/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
/home/tantely/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
/home/tantely/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avo

In [11]:
from gensim.models.wrappers.fasttext import FastText as FT_wrapper
from gensim.test.utils import datapath
ft_home = '/home/tantely/fastText-0.2.0/fasttext'
corpus_file=datapath("/home/tantely/Documents/INSA_Rouen/Projet_fil_rouge/text_mining_fil_rouge/FastTestFolder/xtrain.txt")

model_wrapper = FT_wrapper.train(ft_home, corpus_file)

In [12]:
print(model_wrapper)

FastText(vocab=9357, size=100, alpha=0.025)


## Avantage de fast text:
Peut supporter des mots qui ne se trouvent pas dans notre liste de mots

In [23]:
print("date" in model_wrapper.wv.vocab)
print(model_wrapper['data'])
print("detla" in model_wrapper.wv.vocab)
print(model_wrapper['detla'])

True
[ 2.8440485  -0.3145254   2.9473984  -2.2409658  -1.785316    0.9607362
 -0.2517628   1.1966292   4.514769    3.077068   -4.416133   -3.7850533
  2.833843    5.7973423   2.2054873   4.5341954   2.388404   -1.5108497
  0.28548536  2.0263388   1.0194411  -3.0406046   0.9812421  -4.752426
  0.34504375  1.8508389  -1.4623685   1.7183746   1.1546805   0.01977971
  2.4899638  -1.5656605   3.1972237  -1.1130537  -1.8690281  -1.8412724
  0.7184055   2.5656836  -1.9896026  -1.330899    6.5645022   1.5740051
 -1.162449    4.3562365   0.16167136  0.424391   -1.0717667  -1.3347007
  0.35180995 -1.9359143  -4.4474506  -3.8360472   1.2588416  -1.1305274
  0.08190229 -1.3002462   2.2007837  -1.8586224   0.13748406  2.3846262
  5.456688   -0.4895157  -1.0838895  -0.4587656  -0.7678471   0.09323923
 -1.1486492  -1.9001313   1.5670806  -0.5555484   0.5369114   5.5512257
  0.12485618 -2.1108449   2.6695652   1.3240292   1.8632029  -1.4829787
 -2.466988    1.7188859   1.9109416  -0.33552817  2.781527

On peut alors calculer la similarité de deux mots sans même si l'un des deux mots ne se trouvent pas dans le corpus.

In [25]:
print("data" in model_wrapper.wv.vocab)
print("detla" in model_wrapper.wv.vocab)
model_wrapper.similarity("data", "detla")

True
False


0.19936287

## Opération sur la similarité des mots:

In [27]:
model_wrapper.most_similar("data")

[('sasdata', 0.9555476903915405),
 ('datasets', 0.936911940574646),
 ('databricks', 0.9342582821846008),
 ('etldata', 0.9209719300270081),
 ('exadata', 0.9162241220474243),
 ('metadata', 0.9017711877822876),
 ('voicedata', 0.8847713470458984),
 ('teradata', 0.8515818119049072),
 ('datadriven', 0.8462375402450562),
 ('datastage', 0.8400782346725464)]

In [ ]:
from pyfasttext import FastText
import numpy as np
class FastWord2Vec:
    def __init__(self,inputFile,outputFile,epoch,lr,dim):
        self.inputFile=inputFile
        self.outputFile=outputFile
        self.epoch=epoch
        self.dim=dim
        self.lr=lr
        global model
        
    def __average_word(self,X,modelWords):
        np.array([
            np.mean([self.get_numpy_sentence_vector(w) for w in words.split() if w in modelWords]or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

    def fit(self,X,y=None):
        X.to_csv(self.inputFile,index=False)
        self.model=FastText().cbow(input=self.inputFile,output=self.outputFile,epoch=self.epoch,lr=self.lr,dim=self.dim)
        return self
    def transform(self,X,y):
        return self.__average_word(X,self.words)        
        

In [ ]:
cbowTransformer=FastWord2Vec(inputFile='FastTestFolder/xtrain.txt',outputFile='model',epoch=10,lr=0.5,dim=100)
cbowTransformer.fit(x_train)

/home/tantely/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
